In [ ]:
import time
from awkwardNN.trainAwkwardNet import AwkwardNNTrainer
from awkwardNN.config import get_config
from awkwardNN.utils import print_time
from awkwardNN.preprocessAwkwardData import *

config, _ = get_config()
start = time.time()

np.random.seed(config.random_seed)
torch.manual_seed(config.random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(config.random_seed)

In [ ]:
if config.train:
    trainloader = get_dataloader(
        dataset_size=config.train_size, batch_size=config.batch_size,
        prob_nest=config.prob_nest, prob_signal=config.prob_signal,
        prob_noise=config.prob_noise, max_len=config.max_len,
        max_depth=config.max_depth
    )
    validloader = get_dataloader(
        dataset_size=config.valid_size, batch_size=config.batch_size,
        prob_nest=config.prob_nest, prob_signal=config.prob_signal,
        prob_noise=config.prob_noise, max_len=config.max_len,
        max_depth=config.max_depth
    )
    dataloader = (trainloader, validloader)
else:
    dataloader = get_dataloader(
        dataset_size=config.test_size, batch_size=config.batch_size,
        prob_nest=config.prob_nest, prob_signal=config.prob_signal,
        prob_noise=config.prob_noise, max_len=config.max_len,
        max_depth=config.max_depth
    )

trainer = AwkwardNNTrainer(config, config.max_depth, 1, 2, dataloader)
if config.train:
    trainer.train()
else:
    trainer.test()

print_time(start)

This is a markdown test

$$ x = \sqrt(y)$$
